# 📶 WiFi Setup for Raspberry Pi

Connect your Pi to WiFi easily! After setup, your Pi will automatically connect to WiFi on boot and fall back to hotspot mode if WiFi is unavailable.

## 🚀 Quick Start:

1. **Edit the credentials** in the cell below:
   - Change `WIFI_SSID = ""` to `WIFI_SSID = "YourNetworkName"`
   - Change `WIFI_PASSWORD = ""` to `WIFI_PASSWORD = "YourPassword"`
   - Leave password empty (`""`) for open networks

2. **Run the cell** - it will show available networks first, then connect

3. **Reboot when done** to test automatic connection

## 🔄 How it works:

- **First boot:** Pi starts in hotspot mode (`lectec-ap`)
- **After setup:** Pi tries WiFi first, falls back to hotspot if connection fails
- **Your WiFi:** Saved as `main-wifi` with auto-connect enabled

---

In [ ]:
import subprocess
import time

# ===== EDIT THESE VALUES =====
WIFI_SSID = "YourNetworkName"      # Replace with your WiFi network name
WIFI_PASSWORD = "YourPassword"  # Replace with your password (or leave empty for open networks [no password])
# =============================

def run_cmd(cmd, timeout=30):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)

def check_internet(timeout=10):
    """Check internet connectivity"""
    success, _, _ = run_cmd(f"ping -c 2 -W {timeout} 8.8.8.8")
    return success

def get_hotspot_name():
    """Return the hotspot connection name"""
    return "lectec-ap"

def get_current_hostname():
    """Get current hostname (which matches the hotspot SSID)"""
    success, stdout, _ = run_cmd("hostname")
    if success and stdout.strip():
        return stdout.strip()
    return "ltpi-unknown"

def ensure_hotspot_active():
    """Ensure hotspot is active"""
    hotspot_name = get_hotspot_name()
    current_ssid = get_current_hostname()
    
    print(f"🔄 Ensuring hotspot is active (SSID: {current_ssid})...")
    
    # Check if hotspot is already active
    success, stdout, _ = run_cmd(f"sudo nmcli connection show --active | grep {hotspot_name}")
    if success:
        print(f"✅ Hotspot {current_ssid} already active")
        return True
    
    # Start hotspot
    success, _, stderr = run_cmd(f"sudo nmcli connection up {hotspot_name}")
    if success:
        print(f"✅ Hotspot {current_ssid} activated successfully")
        time.sleep(3)
        return True
    else:
        print(f"❌ Failed to start hotspot {current_ssid}: {stderr}")
        return False

def setup_wifi_connection(ssid, password):
    """Setup or update the main-wifi connection"""
    print(f"🔧 Setting up main-wifi connection for: {ssid}")
    
    # Check if main-wifi connection exists
    success, _, _ = run_cmd("sudo nmcli connection show main-wifi")
    if success:
        print("📝 Updating existing main-wifi connection...")
        # Delete existing connection to avoid conflicts
        run_cmd("sudo nmcli connection delete main-wifi", timeout=10)
        time.sleep(2)
    
    # Create new connection
    if password:
        cmd = f'sudo nmcli dev wifi connect "{ssid}" password "{password}" name main-wifi'
    else:
        cmd = f'sudo nmcli dev wifi connect "{ssid}" name main-wifi'
    
    return run_cmd(cmd, timeout=45)

print("📶 WiFi Setup Starting...")
print("=" * 60)

# Show available networks
print("🔍 Scanning for available WiFi networks...")
success, stdout, _ = run_cmd("sudo nmcli dev wifi list", timeout=15)
if success:
    print("\n📋 Available networks:")
    lines = stdout.strip().split('\n')[1:11]  # Show first 10
    for line in lines:
        if line.strip() and not line.startswith('--'):
            parts = line.split()
            if len(parts) >= 2:
                ssid = parts[1] if parts[1] != '--' else 'Hidden Network'
                signal = parts[7] if len(parts) > 7 else 'Unknown'
                security = 'WPA' if 'WPA' in line else ('Open' if 'WEP' not in line else 'WEP')
                print(f"  • {ssid:25} Signal: {signal:4} Security: {security}")
else:
    print("⚠️  Could not scan networks - proceeding anyway...")

print("\n" + "=" * 60)

# Validate input
if not WIFI_SSID or WIFI_SSID == "Enter Wifi Network Name Here":
    current_ssid = get_current_hostname()
    print("⚠️  Please edit WIFI_SSID and WIFI_PASSWORD above, then run this cell again")
    print("\nInstructions:")
    print("1. Edit WIFI_SSID = \"YourNetworkName\"")
    print("2. Edit WIFI_PASSWORD = \"YourPassword\" (or leave empty for open networks)")
    print("3. Run this cell again")
    
    # Ensure hotspot is running
    ensure_hotspot_active()
    print(f"\n📡 Connect to hotspot: {current_ssid}")
    
else:
    current_ssid = get_current_hostname()
    hotspot_name = get_hotspot_name()
    
    print(f"🔗 Attempting to connect to WiFi: {WIFI_SSID}")
    
    # Step 1: Bring down hotspot
    print("📡 Bringing down hotspot...")
    print(f"\nYou will now lose connection to {current_ssid}. Connect to your main wifi and refresh the page in ~30 seconds.")
    print(f"If it does not connect, the {current_ssid} hotspot should reappear. Connect to it and ensure your SSID and Password are correct before trying again.")
    run_cmd(f"sudo nmcli connection down {hotspot_name}")

    
    # Step 2: Connect to WiFi with proper connection handling
    connection_successful = False
    wifi_error = None
    
    try:
        print(f"🔌 Connecting to {WIFI_SSID}...")
        
        success, stdout, stderr = setup_wifi_connection(WIFI_SSID, WIFI_PASSWORD)
        
        if success:
            print("✅ WiFi connection established!")
            
            # Wait and test internet
            print("⏳ Testing connection...")
            time.sleep(5)
            
            if check_internet():
                connection_successful = True
                print("✅ Internet access confirmed!")
                
                # Show IP
                success, stdout, _ = run_cmd("ip route get 8.8.8.8 | awk '{print $7; exit}'")
                if success and stdout.strip():
                    print(f"📍 WiFi IP: {stdout.strip()}")
            else:
                wifi_error = "Connected to WiFi but no internet access"
                print("⚠️  Connected to WiFi but no internet access")
        else:
            wifi_error = stderr
            print(f"❌ WiFi connection failed: {stderr}")
            
            if "Secrets were required" in stderr:
                print("💡 This usually means the password is incorrect")
            elif "No network with SSID" in stderr:
                print("💡 Network not found - check the SSID spelling")
            elif "already exists" in stderr:
                print("💡 Connection conflict - try running this cell again")
                
    except Exception as e:
        wifi_error = f"Unexpected error: {str(e)}"
        print(f"❌ Unexpected error: {str(e)}")
    
    # Step 3: Handle failure - restore hotspot
    if not connection_successful:
        print("\n🚨 WiFi connection failed - restoring hotspot")
        print("=" * 60)
        
        # Clean up any failed connection attempts
        run_cmd("sudo nmcli connection down main-wifi 2>/dev/null", timeout=10)
        
        # Restore hotspot
        hotspot_restored = ensure_hotspot_active()
        
        if hotspot_restored:
            print(f"\n📡 Hotspot {current_ssid} restored successfully")
            print("🔄 You can now:")
            print("   • Fix the credentials above and try again")
            print("   • Check that the network is available")
            print("   • Verify the password is correct")
            print(f"\n❌ Error: {wifi_error}")
        else:
            print(f"\n🚨 CRITICAL: Could not restore hotspot {current_ssid}!")
            print(f"   Try manually: sudo nmcli connection up {hotspot_name}")
    else:
        # Success!
        print("🎉 WiFi Setup Complete!")


---

# 🔧 Utility Functions

Use the functions below to manage your WiFi connection manually if needed.

## Check Connection Status

Run this to see your current connection status and internet access:

In [ ]:
import subprocess

def run_cmd(cmd):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=15)
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)

print("🔍 Current Connection Status")
print("=" * 40)

# Show active connections
success, stdout, _ = run_cmd("sudo nmcli connection show --active")
if success:
    print("📡 Active connections:")
    for line in stdout.strip().split('\n')[1:]:  # Skip header
        if line.strip():
            parts = line.split()
            if len(parts) >= 4:
                name = parts[0]
                conn_type = parts[2]
                device = parts[3]
                print(f"  • {name:15} ({conn_type}) on {device}")

# Check internet
success, _, _ = run_cmd("ping -c 2 -W 5 8.8.8.8")
if success:
    print("✅ Internet: Working")
else:
    print("❌ Internet: Not working")

# Show IP
success, stdout, _ = run_cmd("ip route get 8.8.8.8 | awk '{print $7; exit}'")
if success and stdout.strip():
    print(f"📍 IP Address: {stdout.strip()}")

## Switch to Hotspot Mode

Run this to manually switch to hotspot mode (lectec-ap):

In [ ]:
import subprocess

def run_cmd(cmd):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=15)
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)

def get_hotspot_name():
    """Return the hotspot connection name"""
    return "lectec-ap"

def get_current_hostname():
    """Get current hostname (which matches the hotspot SSID)"""
    success, stdout, _ = run_cmd("hostname")
    if success and stdout.strip():
        return stdout.strip()
    return "ltpi-unknown"

print("🔄 Switching to hotspot mode...")

hotspot_name = get_hotspot_name()
current_ssid = get_current_hostname()

# Bring down WiFi connections
run_cmd("sudo nmcli connection down main-wifi 2>/dev/null")

# Start hotspot
success, _, stderr = run_cmd(f"sudo nmcli connection up {hotspot_name}")
if success:
    print("✅ Hotspot activated")
    print(f"📡 SSID: {current_ssid}")
    print("🔑 Connect to this network from your device")
else:
    print(f"❌ Failed to start hotspot: {stderr}")


## Switch to WiFi Mode

Run this to manually switch to WiFi (if main-wifi is configured):

In [ ]:
import subprocess
import time

def run_cmd(cmd):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=15)
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)

def get_hotspot_name():
    """Return the hotspot connection name"""
    return "lectec-ap"

def get_current_hostname():
    """Get current hostname (which matches the hotspot SSID)"""
    success, stdout, _ = run_cmd("hostname")
    if success and stdout.strip():
        return stdout.strip()
    return "ltpi-unknown"

print("🔄 Switching to WiFi...")

hotspot_name = get_hotspot_name()
current_ssid = get_current_hostname()

# Check if main-wifi exists
success, _, _ = run_cmd("sudo nmcli connection show main-wifi")
if not success:
    print("❌ main-wifi connection not configured. Use the setup above first.")
else:
    # Bring down hotspot
    run_cmd(f"sudo nmcli connection down {hotspot_name} 2>/dev/null")
    
    # Start WiFi
    success, _, stderr = run_cmd("sudo nmcli connection up main-wifi")
    if success:
        print("✅ WiFi activated")
        
        # Test internet
        time.sleep(3)
        success, _, _ = run_cmd("ping -c 2 -W 5 8.8.8.8")
        if success:
            print("✅ Internet access confirmed")
        else:
            print("⚠️  WiFi connected but no internet")
    else:
        print(f"❌ Failed to connect to WiFi: {stderr}")
        print("🔄 Restoring hotspot...")
        run_cmd(f"sudo nmcli connection up {hotspot_name}")
        print(f"📡 Hotspot {current_ssid} restored")
